In [17]:
import pandas as pd

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('x_test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Separate features and target variable from training data
X = train_data.drop(['id', 'y'], axis=1)
y = train_data['y']

# Split the data into training and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

X_train_scaled.shape, X_val_scaled.shape


((128, 64), (32, 64))

In [19]:
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

# Initialize Lasso regression model
lasso_model = Lasso()

# Train the model
lasso_model.fit(X_train_scaled, y_train)

# Predict on validation set
y_val_pred = lasso_model.predict(X_val_scaled)

# Calculate mean squared error on validation set
mse = mean_squared_error(y_val, y_val_pred)
mse

4345.423238344814

In [20]:
from sklearn.model_selection import GridSearchCV
import numpy as np

# Define the range of hyperparameters to search
param_grid = {
    'alpha': np.logspace(-4, 4, 20)
}

# Use GridSearchCV to find the best hyperparameters
lasso_search = GridSearchCV(Lasso(), param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
lasso_search.fit(X_train_scaled, y_train)

# Get the best hyperparameters and the corresponding mean squared error
best_alpha = lasso_search.best_params_['alpha']
best_mse = -lasso_search.best_score_

best_alpha, best_mse


/Users/mihir/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.636e+04, tolerance: 5.719e+01
  model = cd_fast.enet_coordinate_descent(
/Users/mihir/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.702e+04, tolerance: 5.689e+01
  model = cd_fast.enet_coordinate_descent(
/Users/mihir/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duali

(4.281332398719396, 3054.8060238646767)

In [22]:
# Train the Lasso model with the optimal alpha value
optimized_lasso = Lasso(alpha=best_alpha)
optimized_lasso.fit(X_train_scaled, y_train)

# Predict on validation set using the optimized model
y_val_pred_optimized = optimized_lasso.predict(X_val_scaled)

# Calculate mean squared error on validation set for the optimized model
mse_optimized = mean_squared_error(y_val, y_val_pred_optimized)
mse_optimized

4097.427578620105

In [23]:
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LinearRegression

# Use Recursive Feature Elimination with Cross-Validation
selector = RFECV(LinearRegression(), step=1, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
selector = selector.fit(X_train_scaled, y_train)

# Select important features based on RFE
X_train_selected = X_train_scaled[:, selector.support_]
X_val_selected = X_val_scaled[:, selector.support_]

X_train_selected.shape

(128, 6)

In [24]:
# Train the Lasso model with the optimal alpha value on the selected features
optimized_lasso_selected = Lasso(alpha=best_alpha)
optimized_lasso_selected.fit(X_train_selected, y_train)

# Predict on validation set using the optimized model with selected features
y_val_pred_selected = optimized_lasso_selected.predict(X_val_selected)

# Calculate mean squared error on validation set for the optimized model with selected features
mse_selected = mean_squared_error(y_val, y_val_pred_selected)
mse_selected

4926.062340631568

In [25]:
from sklearn.ensemble import GradientBoostingRegressor

# Initialize a basic Gradient Boosting Regressor
gbm = GradientBoostingRegressor(random_state=42)

# Train the model
gbm.fit(X_train_scaled, y_train)

# Predict on the validation set
y_val_pred_gbm = gbm.predict(X_val_scaled)

# Calculate mean squared error for the basic Gradient Boosting Regressor
mse_gbm = mean_squared_error(y_val, y_val_pred_gbm)
mse_gbm

3695.3443360718306

In [26]:
# Define the range of hyperparameters to search for Gradient Boosting Regressor
param_grid_gbm = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.05, 0.1, 0.5],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 3, 4],
    'min_samples_leaf': [1, 2, 3],
    'subsample': [0.8, 0.9, 1.0]
}

# Use GridSearchCV to find the best hyperparameters for Gradient Boosting Regressor
gbm_search = GridSearchCV(GradientBoostingRegressor(random_state=42), param_grid_gbm, 
                          cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)

gbm_search.fit(X_train_scaled, y_train)

# Get the best hyperparameters and the corresponding mean squared error
best_params_gbm = gbm_search.best_params_
best_mse_gbm = -gbm_search.best_score_

best_params_gbm, best_mse_gbm


Fitting 5 folds for each of 972 candidates, totalling 4860 fits


({'learning_rate': 0.05,
  'max_depth': 3,
  'min_samples_leaf': 2,
  'min_samples_split': 2,
  'n_estimators': 50,
  'subsample': 1.0},
 3310.024626648622)

In [27]:
# Train the Gradient Boosting Regressor with the optimized hyperparameters
optimized_gbm = GradientBoostingRegressor(
    learning_rate=0.05,
    max_depth=3,
    min_samples_leaf=2,
    min_samples_split=2,
    n_estimators=50,
    subsample=1.0,
    random_state=42
)

optimized_gbm.fit(X_train_scaled, y_train)

# Predict on the validation set using the optimized model
y_val_pred_optimized_gbm = optimized_gbm.predict(X_val_scaled)

# Calculate mean squared error for the optimized Gradient Boosting Regressor
mse_optimized_gbm = mean_squared_error(y_val, y_val_pred_optimized_gbm)
mse_optimized_gbm


3480.028037158167

In [31]:
# Scale the test data using the previously defined scaler
x_test_scaled = scaler.transform(test_data.drop('id', axis=1))

# Predict diabetic levels for the test data using the optimized Gradient Boosting Regressor
y_test_pred = optimized_gbm.predict(x_test_scaled)

# Create a DataFrame for the submission
submission_df = pd.DataFrame({
    'id': test_data['id'],
    'y': y_test_pred
})

# Save the submission DataFrame to a CSV file
submission_path = 'explore_submission.csv'
submission_df.to_csv(submission_path, index=False)

submission_path

'explore_submission.csv'

Nothing next needed

In [ ]:
import xgboost as xgb

# Initialize a basic XGBoost regressor
xgb_regressor = xgb.XGBRegressor(objective ='reg:squarederror', random_state=42)

# Train the model
xgb_regressor.fit(X_train_scaled, y_train)

# Predict on the validation set
y_val_pred_xgb = xgb_regressor.predict(X_val_scaled)

# Calculate mean squared error for the basic XGBoost regressor
mse_xgb = mean_squared_error(y_val, y_val_pred_xgb)
mse_xgb


4847.6910748298305

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Define hyperparameters for XGBoost
param_dist = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.01, 0.05, 0.1, 0.5],
    'max_depth': [3, 4, 5, 6],
    'min_child_weight': [1, 2, 3],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

# Use RandomizedSearchCV for hyperparameter optimization
xgb_search = RandomizedSearchCV(
    xgb.XGBRegressor(objective='reg:squarederror', random_state=42),
    param_distributions=param_dist,
    n_iter=50,
    scoring='neg_mean_squared_error',
    cv=5,
    n_jobs=-1,
    verbose=1,
    random_state=42
)

xgb_search.fit(X_train_scaled, y_train)

# Get the best hyperparameters and the corresponding mean squared error
best_params_xgb = xgb_search.best_params_
best_mse_xgb = -xgb_search.best_score_

best_params_xgb, best_mse_xgb


Fitting 5 folds for each of 50 candidates, totalling 250 fits


({'subsample': 1.0,
  'n_estimators': 200,
  'min_child_weight': 2,
  'max_depth': 3,
  'learning_rate': 0.01,
  'colsample_bytree': 0.9},
 3545.4923269698716)

In [ ]:
# Train the XGBoost regressor with the optimized hyperparameters
optimized_xgb = xgb.XGBRegressor(
    subsample=1.0,
    n_estimators=200,
    min_child_weight=2,
    max_depth=3,
    learning_rate=0.01,
    colsample_bytree=0.9,
    objective='reg:squarederror',
    random_state=42
)

optimized_xgb.fit(X_train_scaled, y_train)

# Predict on the validation set using the optimized model
y_val_pred_optimized_xgb = optimized_xgb.predict(X_val_scaled)

# Calculate mean squared error for the optimized XGBoost regressor
mse_optimized_xgb = mean_squared_error(y_val, y_val_pred_optimized_xgb)
mse_optimized_xgb


3648.1118834584686

In [ ]:
# Scale the test data
X_test_scaled = scaler.transform(X_test)

# Predict using the optimized XGBoost regressor
y_test_pred = optimized_xgb.predict(X_test_scaled)

# Create the submission dataframe
submission_df = pd.DataFrame({
    'id': test_ids,
    'y': y_test_pred
})

submission_df.head()


NameError: name 'X_test' is not defined